# Stop loss & Take profit with vectorbt

- Reference 1: https://quantnomad.com/using-sl-and-pt-in-backtesting-in-python-with-vectrobt/
- Reference 2: https://medium.datadriveninvestor.com/stop-loss-trailing-stop-or-take-profit-2-million-backtests-shed-light-dde23bda40be & https://github.com/polakowo/vectorbt/blob/master/examples/StopSignals.ipynb
- Reference 3: http://qubitquants.pro/customsim_2/index.html

1. Basic stop loss & take profit by setting % relative to entry price

In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
from numba import njit
import talib
import datetime as dt
import time
from collections import namedtuple
import itertools
import math
from numba import njit


In [2]:
yfdata = vbt.YFData.download(
    ["AAPL","MSFT"],
    missing_index='drop',
    interval = '1d'
)
ohlcv = yfdata.concat()
 
fast_ma = vbt.MA.run(ohlcv["Close"], 10, short_name='fast MA')
slow_ma = vbt.MA.run(ohlcv["Close"], 50, short_name='slow MA')

long_entries = fast_ma.ma_crossed_above(slow_ma)
short_entries = fast_ma.ma_crossed_below(slow_ma)

C:\Users\yongsheng.tan\AppData\Local\miniconda3\envs\vectorbt\lib\site-packages\vectorbt\data\base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)


In [3]:
pf = vbt.Portfolio.from_signals(
    ohlcv["Close"],
    entries = long_entries,
    short_entries = short_entries,
    sl_stop=0.025, # in % relative to entry price
    tp_stop=0.05, # in % relative to entry price
    freq = "1D",
    upon_opposite_entry='close',
    group_by=False
)

In [11]:
ohlcv = {
    k: v.vbt.split_into_ranges(range_len=window_len.days, n=window_count) 
    for k, v in ohlcv.items()
}
    
print(ohlcv['Open'].shape)

AttributeError: 'Vbt_DFAccessor' object has no attribute 'split_into_ranges'

In [4]:
pf.wrapper.columns

MultiIndex([(10, 50, 'AAPL'),
            (10, 50, 'MSFT')],
           names=['fast MA_window', 'slow MA_window', 'symbol'])

In [5]:
pf[(10, 50, 'AAPL')].stats()

Start                         1986-03-13 05:00:00+00:00
End                           2023-08-18 04:00:00+00:00
Period                               9435 days 00:00:00
Start Value                                       100.0
End Value                                    487.962312
Total Return [%]                             387.962312
Benchmark Return [%]                      203712.127472
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              31.128734
Max Drawdown Duration                2219 days 00:00:00
Total Trades                                        215
Total Closed Trades                                 214
Total Open Trades                                     1
Open Trade PnL                                 8.736217
Win Rate [%]                                  45.794393
Best Trade [%]                                16.746493
Worst Trade [%]                              -12

In [10]:
entries = pd.DataFrame.vbt.signals.empty_like(ohlcv['Open'])
entries.iloc[0, :] = True

entries.to_csv("test.csv")